In [2]:
import numpy as np
import os
import pandas as pd

from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker .config import ExportFormat
from tflite_model_maker .text_classifier import AverageWordVecSpec
from tflite_model_maker .text_classifier import DataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

In [3]:
import tensorflow as tf

print(len(tf.config.list_physical_devices('GPU')))

0


In [4]:
df = pd.read_csv('emotion_dataset.csv')

label_map = {0: 'neutral', 1: 'joy', 2: 'sadness', 3: 'angry', 4:'fear'}

df = df.loc[ (df['Emotion'] == 'sadness') | (df['Emotion'] =='fear') | (df['Emotion'] == 'anger')| (df['Emotion'] == 'joy')| (df['Emotion'] == 'neutral')]
print(df['Emotion'].value_counts())
#df=df.drop(df.query("Emotion == 'joy'").sample(frac = 0.5).index)
print(df['Emotion'].value_counts())
df=df.drop(['Text', 'Unnamed: 0'], axis = 1)
df['Clean_Text'] = df['Clean_Text'].str.lower()
df.head()



joy        11045
sadness     6722
fear        5410
anger       4297
neutral     2254
Name: Emotion, dtype: int64
joy        11045
sadness     6722
fear        5410
anger       4297
neutral     2254
Name: Emotion, dtype: int64


,Emotion,Clean_Text
0,neutral,NaN
1,joy,sage act upgrade list tommorow
2,sadness,way homegirl baby funeral man hate funerals sh...
3,joy,eye true hazel eyeand brilliant regular feat...
4,joy,ugh babe hugggzzz u babe naamazed nga ako e...


In [5]:
train = df.sample(frac = 0.8, random_state = 42)
test = df.loc[~df.index.isin(train.index)]

In [6]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [8]:
spec = model_spec.get('average_word_vec')
train_data = DataLoader.from_csv(filename = 'train.csv', text_column = 'Clean_Text', label_column = 'Emotion', model_spec = spec, is_training = True)
test_data = DataLoader.from_csv(filename = 'test.csv', text_column = 'Clean_Text', label_column = 'Emotion', model_spec = spec, is_training = False)


In [ ]:
import tflite_model_maker
tflite_model_maker.config.QuantizationConfig(inference_input_type=tf.float32, inference_output_type=tf.float32)

In [9]:
model = text_classifier.create(train_data, model_spec = spec, epochs = 80)

Epoch 1/80
743/743 [==============================] - 1s 1ms/step - loss: 1.5027 - accuracy: 0.3686
Epoch 2/80
743/743 [==============================] - 1s 1ms/step - loss: 1.4906 - accuracy: 0.3695
Epoch 3/80
743/743 [==============================] - 1s 1ms/step - loss: 1.4744 - accuracy: 0.3700
Epoch 4/80
743/743 [==============================] - 1s 1ms/step - loss: 1.4349 - accuracy: 0.3856
Epoch 5/80
743/743 [==============================] - 1s 1ms/step - loss: 1.3677 - accuracy: 0.4269
Epoch 6/80
743/743 [==============================] - 1s 1ms/step - loss: 1.3044 - accuracy: 0.4497
Epoch 7/80
743/743 [==============================] - 1s 1ms/step - loss: 1.2512 - accuracy: 0.4815
Epoch 8/80
743/743 [==============================] - 1s 1ms/step - loss: 1.2071 - accuracy: 0.5154
Epoch 9/80
743/743 [==============================] - 1s 1ms/step - loss: 1.1676 - accuracy: 0.5348
Epoch 10/80
743/743 [==============================] - 1s 1ms/step - loss: 1.1297 - accuracy: 0.5536

In [10]:
loss, acc = model.evaluate(test_data)

186/186 [==============================] - 1s 4ms/step - loss: 0.9605 - accuracy: 0.6749


In [12]:
print(type(train_data))
print(vars(train_data))
print(train_data[0])
print(test_data[0])

<class 'tensorflow_examples.lite.model_maker.core.data_util.text_dataloader.TextClassifierDataLoader'>
{'_dataset': <ParallelMapDataset shapes: ((256,), ()), types: (tf.int32, tf.int32)>, '_size': 23782, 'index_to_label': ['anger', 'fear', 'joy', 'neutral', 'sadness']}


TypeError: 'TextClassifierDataLoader' object is not subscriptable